In [42]:
import pandas as pd
import numpy as np
import random

In [43]:
# Generate Timestamps from January 2018 to December 2021 with hourly frequency
start_date = '2018-01-01'
end_date = '2021-12-31'
timestamps = pd.date_range(start=start_date, end=end_date, freq='H')
np.random.seed(42)

In [80]:
# Generate CPUTime: Mean = 350000, with fluctuations based on time of day, holidays, and random spikes
def generate_cputime(timestamps):
    # Seasonal pattern with drops during Christmas holidays, exams, and summer months
    holiday_pattern =1 - np.isin(timestamps.month, [12, 6, 7 ,8, 9])*0.1

    # Random spikes
    spike_interval = 24 * 7
    
    spikes = 2.5 - 1.5*np.abs((timestamps.hour % 6 ) / spike_interval)

    # Noise
    noise = 1 + np.random.normal(loc=0, scale=0.01, size=len(timestamps))

    #cycle
    cycle = np.sin(np.arange(0, len(timestamps)) * 2 * np.pi / 24) * 0.1 + 1

    series = 1000000 * noise * holiday_pattern * cycle * spikes

    return np.clip(series, 0, None)

In [105]:
# Generate MemoryUsed: High correlation with CPUTime but non-linear relationship
def generate_memoryused(cputime):
    # Add some noise and non-linear relationship to CPUTime
    noise = np.random.normal(loc=0, scale=2000, size=len(cputime))

    memory_ratio = (np.sin(cputime / 250000) * 20) + 100

    return np.clip(cputime * memory_ratio + noise, 0, None)

In [106]:
# Generate NJobs: Non-linear relationship with CPUTime and MemoryUsed
def generate_nprocs(cputime, memoryused):
    # Non-linear relationship with CPUTime and MemoryUsed
    cputime = np.clip(cputime, 1e-8, None)
    cputime_factor = np.clip(np.log(cputime), -10, None) 
    memoryused_factor = np.sqrt(memoryused)*np.tanh(memoryused / 1e9) / 10

    # Random fluctuations with some correlation to CPUTime and MemoryUsed
    njobs_noise = np.random.normal(loc=15, scale=10, size=len(cputime))
    njobs = cputime_factor + memoryused_factor + njobs_noise

    return np.clip(njobs, 1, 600)  # Clip to a minimum of 1 and maximum of 100 jobs


In [107]:
def generate_njobs(nprocs):
    njobs_random_factor = np.random.normal(loc=6, scale=0.5, size=len(nprocs))

    njobs = nprocs / np.clip(njobs_random_factor, 1, None)
    return njobs 

In [108]:
def generate_jobs(timestamps):
    arrived_jobs = np.random.poisson(lam=10, size=len(timestamps))
    completed_jobs = np.random.poisson(lam=8, size=len(timestamps))
    return arrived_jobs, completed_jobs

In [109]:
# Generate the synthetic dataset
cputime = generate_cputime(timestamps)
memoryused = generate_memoryused(cputime)
nprocs = generate_nprocs(cputime, memoryused)
njobs = generate_njobs(nprocs)
arrived_jobs, completed_jobs = generate_jobs(timestamps)

In [110]:
# Combine all features into a DataFrame
data = {
    'Timestamp': timestamps,
    'CPUTime': cputime,
    'Memory': memoryused,
    'NJobs': njobs,
    'NProcs': nprocs,
    'ArrivedJobs': arrived_jobs,
    'CompletedJobs': completed_jobs,
}
dataset = pd.DataFrame(data)

dataset["cosHour"] = dataset["Timestamp"].map(lambda timestamp: np.cos(2*np.pi*timestamp.hour/24))
dataset["sinHour"] = dataset["Timestamp"].map(lambda timestamp: np.sin(2*np.pi*timestamp.hour/24))

dataset["cosDay"] = dataset["Timestamp"].map(lambda timestamp: np.cos(2*np.pi*timestamp.timetuple().tm_yday/365))
dataset["sinDay"] = dataset["Timestamp"].map(lambda timestamp: np.sin(2*np.pi*timestamp.timetuple().tm_yday/365))

dataset["cosMonth"] = dataset["Timestamp"].map(lambda timestamp: np.cos(2*np.pi*timestamp.month/12))
dataset["sinMonth"] = dataset["Timestamp"].map(lambda timestamp: np.sin(2*np.pi*timestamp.month/12))


dataset.to_csv("./datasets/final/final-formed-synthetic.csv", index=False)

In [111]:
dataset.head()

,Timestamp,CPUTime,Memory,NJobs,NProcs,ArrivedJobs,CompletedJobs,cosHour,sinHour,cosDay,sinDay,cosMonth,sinMonth
0,2018-01-01 00:00:00,2.537043e+06,2.201173e+08,55.213717,345.328568,10,13,1.000000,0.000000,0.999852,0.017213,0.866025,0.5
1,2018-01-01 01:00:00,2.504147e+06,2.224755e+08,66.379790,357.240027,8,5,0.965926,0.258819,0.999852,0.017213,0.866025,0.5
2,2018-01-01 02:00:00,2.627551e+06,2.162752e+08,59.001732,354.975088,8,10,0.866025,0.500000,0.999852,0.017213,0.866025,0.5
3,2018-01-01 03:00:00,2.660781e+06,2.161338e+08,52.598819,327.892378,10,6,0.707107,0.707107,0.999852,0.017213,0.866025,0.5
4,2018-01-01 04:00:00,2.696776e+06,2.169104e+08,52.523992,338.741574,11,9,0.500000,0.866025,0.999852,0.017213,0.866025,0.5


In [112]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CPUTime,35041.0,2.373944e+06,2.095863e+05,1.927694e+06,2.239128e+06,2.360283e+06,2.532418e+06,2.831284e+06
Memory,35041.0,2.326273e+08,1.082526e+07,2.160869e+08,2.211944e+08,2.352332e+08,2.427738e+08,2.465867e+08
NJobs,35041.0,6.356813e+01,6.911450e+00,4.294669e+01,5.869460e+01,6.320315e+01,6.800503e+01,9.871066e+01
NProcs,35041.0,3.785226e+02,2.568877e+01,3.092486e+02,3.557978e+02,3.826396e+02,3.999177e+02,4.439747e+02
ArrivedJobs,35041.0,1.002152e+01,3.168426e+00,0.000000e+00,8.000000e+00,1.000000e+01,1.200000e+01,2.400000e+01
CompletedJobs,35041.0,8.014697e+00,2.826395e+00,0.000000e+00,6.000000e+00,8.000000e+00,1.000000e+01,2.500000e+01
cosHour,35041.0,2.853800e-05,7.071270e-01,-1.000000e+00,-7.071068e-01,6.123234e-17,7.071068e-01,1.000000e+00
sinHour,35041.0,-1.163420e-17,7.071068e-01,-1.000000e+00,-7.071068e-01,0.000000e+00,7.071068e-01,1.000000e+00
cosDay,35041.0,2.843652e-05,7.071268e-01,-9.999630e-01,-7.086267e-01,4.303538e-03,7.025275e-01,1.000000e+00
sinDay,35041.0,1.178963e-05,7.071069e-01,-9.999907e-01,-7.055836e-01,6.432491e-16,7.055836e-01,9.999907e-01
